# langchainマニュアルのgemini版
https://python.langchain.com/v0.1/docs/expression_language/primitives/passthrough/#retrieval-example

In [1]:

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # type: ignore
chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0) # type: ignore #,convert_system_message_to_human=True,temperature=0)



vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=gemini_embeddings
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
# model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough(passed=RunnablePassthrough())}
    | prompt
    | chat
    | StrOutputParser()
)

result = retrieval_chain.invoke("where did harrison work?")

In [2]:
print(type(result))

<class 'str'>


In [3]:
print(result)

Harrison worked at Kensho. 



In [11]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed2=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

result = runnable.invoke({"num": 1})

In [12]:
print(type(result))
print(result.keys())

<class 'dict'>
dict_keys(['passed2', 'modified'])


In [4]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
# from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model  = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # type: ignore

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [5]:
result = chain.invoke({"foo": "bar", "bar": "gah"})
print(type(result))

<class 'langchain_core.messages.ai.AIMessage'>


In [7]:
print(result.content)

3 + 9 = 12 



In [12]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

model  = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # type: ignore

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)


chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

lang = "japanese"
result = chain2.invoke({"person": "obama", "language": lang})

In [13]:
print(type(result))
print(result)

<class 'str'>
バラク・オバマが生まれたホノルルは、**アメリカ合衆国**にあります。 



In [34]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

results = {}

runnable1 = RunnableParallel(
    color=RunnablePassthrough(),

    
)

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | runnable1| {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [35]:
result = question_generator.invoke("sharp")

In [38]:
print(type(result))
print(result)

<class 'langchain_core.prompt_values.ChatPromptValue'>
messages=[HumanMessage(content='What is the color of Blueberry \n and the flag of Dominica \n?')]


# その他

## StrOutputParser
StrOutputParser無しのresult.content = StrOutputPasser有りのresult

チェーン内で使う

In [1]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # type: ignore
chatm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0) # type: ignore #,convert_system_message_to_human=True,temperature=0)

In [15]:
from langchain_core.output_parsers import StrOutputParser

In [4]:
text = "Hola, ¿cómo estás? Espero que todo vaya bien."
# text = spanish_email

In [5]:
system_template="you are a language master who can use many languages"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
system_message_prompt.input_variables


[]

In [6]:
# human_template = "please tell me what language is used in \n{text}\nI want you answer just the name of the lanugage only."
human_template = "please tell me what language is used in \n{text}\n"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
human_message_prompt.input_variables

['text']

In [7]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

In [8]:
chat_prompt.input_variables
request = chat_prompt.format_prompt(text=text).to_messages()

In [ ]:
chat= ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True)
result = chat.invoke(request)

In [26]:
print(result)

content='The language used in "Hola, ¿cómo estás? Espero que todo vaya bien." is **Spanish**. \n\nLet me break it down for you:\n\n* **Hola:**  This is a common Spanish greeting, equivalent to "Hello" in English.\n* **¿cómo estás?**: This is a question meaning "How are you?"\n* **Espero:** This means "I hope"\n* **que todo vaya bien:** This translates to "that everything goes well"\n\nSo the entire phrase is a friendly greeting in Spanish, wishing someone well. \n' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-8bb56cca-ce59-47e2-9818-4

In [27]:
print(result.content)

The language used in "Hola, ¿cómo estás? Espero que todo vaya bien." is **Spanish**. 

Let me break it down for you:

* **Hola:**  This is a common Spanish greeting, equivalent to "Hello" in English.
* **¿cómo estás?**: This is a question meaning "How are you?"
* **Espero:** This means "I hope"
* **que todo vaya bien:** This translates to "that everything goes well"

So the entire phrase is a friendly greeting in Spanish, wishing someone well. 



In [ ]:
chain = chat  | StrOutputParser()
result = chain.invoke(request)

In [29]:
print(result)

The language used in "Hola, ¿cómo estás? Espero que todo vaya bien." is **Spanish**. 

Let me break it down for you:

* **Hola:**  This is the Spanish word for "Hello".
* **¿cómo estás?:** This translates to "How are you?"
* **Espero:** This means "I hope".
* **que todo vaya bien:** This phrase means "that everything is going well". 

Let me know if you'd like me to translate something else! 😊 



# geminiでのagent
https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/

In [48]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate


In [49]:
@tool
def magic_function(input: str) -> str:
    """Applies a magic function to an input."""
    return "LangChain is an open-source Python library that helps developers build applications with large language models (LLMs) through composability. Some key features of LangChain include:\n\n - Retrieval augmented generation - Allowing LLMs to retrieve and utilize external data sources when generating outputs.\n - Analyzing structured data - Tools for working with structured data like databases, APIs, PDFs, etc. and allowing LLMs to reason over this data.\n - Building chatbots and agents - Frameworks for building conversational AI applications.\n - Composability - LangChain allows you to chain together different LLM capabilities and data sources in a modular and reusable way.\n The library aims to make it easier to build real-world applications that leverage the power of large language models in a scalable and robust way. It provides abstractions and primitives for working with LLMs from different providers like OpenAI, Anthropic, Cohere, etc. LangChain is open-source and has an active community contributing new features and improvements."

tools = [magic_function]

# tools = [TavilySearchResults(max_results=1)]

In [50]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True)

In [51]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [56]:
# Create an agent executor by passing in the agent and tools
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,return_intermediate_steps=True)
result = agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...


/home/hiroshisakuma/projects/.py310env/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `magic_function` with `{'input': 'what is LangChain?'}`


LangChain is an open-source Python library that helps developers build applications with large language models (LLMs) through composability. Some key features of LangChain include:

 - Retrieval augmented generation - Allowing LLMs to retrieve and utilize external data sources when generating outputs.
 - Analyzing structured data - Tools for working with structured data like databases, APIs, PDFs, etc. and allowing LLMs to reason over this data.
 - Building chatbots and agents - Frameworks for building conversational AI applications.
 - Composability - LangChain allows you to chain together different LLM capabilities and data sources in a modular and reusable way.
 The library aims to make it easier to build real-world applications that leverage the power of large language models in a scalable and robust way. It provides abstractions and primitives for working with LLMs from different providers like OpenAI, Anthropic,

/home/hiroshisakuma/projects/.py310env/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


LangChain is an open-source Python library that helps developers build applications with large language models (LLMs) through composability. Some key features of LangChain include:

 - Retrieval augmented generation - Allowing LLMs to retrieve and utilize external data sources when generating outputs.
 - Analyzing structured data - Tools for working with structured data like databases, APIs, PDFs, etc. and allowing LLMs to reason over this data.
 - Building chatbots and agents - Frameworks for building conversational AI applications.
 - Composability - LangChain allows you to chain together different LLM capabilities and data sources in a modular and reusable way.

The library aims to make it easier to build real-world applications that leverage the power of large language models in a scalable and robust way. It provides abstractions and primitives for working with LLMs from different providers like OpenAI, Anthropic, Cohere, etc. LangChain is open-source and has an active community co

In [57]:
print(type(result))
print(result.keys())

<class 'dict'>
dict_keys(['input', 'output', 'intermediate_steps'])


In [58]:
print(result["input"])

what is LangChain?


In [59]:
print(result["output"])


LangChain is an open-source Python library that helps developers build applications with large language models (LLMs) through composability. Some key features of LangChain include:

 - Retrieval augmented generation - Allowing LLMs to retrieve and utilize external data sources when generating outputs.
 - Analyzing structured data - Tools for working with structured data like databases, APIs, PDFs, etc. and allowing LLMs to reason over this data.
 - Building chatbots and agents - Frameworks for building conversational AI applications.
 - Composability - LangChain allows you to chain together different LLM capabilities and data sources in a modular and reusable way.

The library aims to make it easier to build real-world applications that leverage the power of large language models in a scalable and robust way. It provides abstractions and primitives for working with LLMs from different providers like OpenAI, Anthropic, Cohere, etc. LangChain is open-source and has an active community co

In [60]:
print(result["intermediate_steps"])


[(ToolAgentAction(tool='magic_function', tool_input={'input': 'what is LangChain?'}, log="\nInvoking: `magic_function` with `{'input': 'what is LangChain?'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'magic_function', 'arguments': '{"input": "what is LangChain?"}'}}, response_metadata={'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-1dc80ec2-f01f-4703-9a77-44a64875654c', tool_calls=[{'name': 'magic_function', 'args': {'input': 'what is LangChain?'}, 'id': 'd1551aad-1e28-43d2-9a8b-bac5b2400a61', 'type': 'tool_call'}], usage_metadata={'input_tokens': 69, 'output_token

# intermediate result試行

In [2]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)


# model = ChatOpenAI()
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

/home/hiroshisakuma/projects/.py310env/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/home/hiroshisakuma/projects/.py310env/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Honolulu, Hawái, está en **Estados Unidos**. \n'

In [27]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableParallel


prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)



model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True)
llm_runnable = RunnableParallel(passed=RunnablePassthrough(), output=model)


# chain1 = prompt1 | model | StrOutputParser()
# chain1 = prompt1 | llm_runnable | StrOutputParser()
chain1 = prompt1 | llm_runnable 
result = chain1.invoke({"person":"obama", "language":"english"})

# chain2 = (
#     {"city": chain1, "language": itemgetter("language")}
#     | prompt2
#     | model
#     | StrOutputParser()
# )

# result = chain2.invoke({"person": "obama", "language": "english"})

/home/hiroshisakuma/projects/.py310env/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [28]:
print(type(result))

<class 'dict'>


In [29]:
print(result.keys())

dict_keys(['passed', 'output'])
